In [1]:
# Import essential packages
import pandas as pd
import numpy as np
import time
import re
import bs4

from bs4 import BeautifulSoup as soup
from datetime import datetime, date
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

clean = True

# Gather all the access
path = 'C:\Program Files (x86)\chromedriver.exe'
source = 'http://covid19.bangka.go.id/#'

opsi = webdriver.ChromeOptions()
opsi.add_argument('--no-sandbox')
opsi.add_argument('--disable-dev-shm-usage')
opsi.add_argument("--start-maximized")
opsi.add_argument("--incognito")
opsi.add_argument("--headless")

In [4]:
from webdriver_manager.chrome import ChromeDriverManager
# Get the access
# driver = webdriver.Chrome(path, options = opsi)
driver = webdriver.Chrome(ChromeDriverManager(version='95.0.4638.69').install(), options = opsi)
driver.get(source)
content = driver.page_source
page_html = soup(content, 'html.parser')



====== WebDriver manager ======
Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.69/chromedriver_mac64.zip
Driver has been saved in cache [/Users/gitasafitri/.wdm/drivers/chromedriver/mac64/95.0.4638.69]


In [5]:
# Define table and date information
table_html = page_html.find('table')
date_html = page_html.find('h4', {'class': 'text-center'})

In [6]:
# Gather the date
today = datetime.now()
scrape_date = '%s' % (date.today().strftime('%Y-%m-%d'))

dateupd = date_html.text.strip()
update_match = re.search(r"\d{4}-\d{1,}-\d{1,}", dateupd)
update_str = update_match.group(0)
date = datetime.strptime(update_str, '%Y-%m-%d')
date_update = '%s' % (date.strftime('%Y-%m-%d'))

In [7]:
# Gather all the data
container = table_html.findAll('tr')

del container[0]

dat1 = []
dat2 = []
dat3 = []

for area in container:
    area_container = area.findAll('td')
    for td in area_container:
        strong = td.findAll('strong')
        for i in range(0, len(strong)):
            total = [strong[i].text.strip()]
            total = [int(j) for j in total]
            dat1.append(total)
#             print(total)
    for sup in area_container:
        area_sup = sup.findAll('sup')
        for i in range(0, len(area_sup)):
            tambah = [area_sup[i].text.strip()]
            tambah = [int(k) for k in tambah]
            dat2.append(tambah)
#             print(tambah)
    for j in range(0, len(area_container)):
        final_kec = [area_container[j].text.lower().strip()]
        for m in range(len(final_kec)):
            if bool(re.search("[a-z]+", final_kec[m])):
                nama = final_kec[m]
                dat3.append(nama)

total_num = np.reshape(dat1, (-1, 4))
tambah_num = np.reshape(dat2, (-1,3))

In [8]:
driver.quit()

In [10]:
data = pd.DataFrame(total_num)
data.insert(0, column = 'kecamatan', value = dat3)
data1 = pd.DataFrame(tambah_num)
data1.insert(0, column = 'kecamatan', value = dat3)

df = data.merge(data1, left_on = 'kecamatan', right_on = 'kecamatan')
df.rename(columns = {'0_x': 'total_positif',
                       '2_x': 'positif_meninggal',
                       '0_y': 'positif_isolasi',
                       '1_y': 'positif_sembuh',
                    3: 'positif_dirawat'}, inplace = clean)

df = df.drop(df.columns[[2, 7]], axis = 1)

df.insert(0, column = 'scrape_date', value = scrape_date)
df.insert(1, column = 'date_update', value = date_update)
df.insert(2, column = 'provinsi', value = 'kepulauan bangka belitung')
df.insert(3, column = 'kabkot', value = 'bangka')
df.insert(5, column = 'kelurahan', value = '')
df.insert(6, column = 'total_odp', value = '')
df.insert(7, column = 'total_pdp', value = '')
df.insert(9, column = 'source_link', value = source)
df.insert(10, column = 'types', value = 'kecamatan')
df.insert(11, column = 'user_pic', value = 'Tubagus')

df = df[['scrape_date','date_update','provinsi','kabkot','kecamatan','kelurahan','total_odp','total_pdp','total_positif',
            'positif_sembuh','positif_dirawat','positif_isolasi','positif_meninggal','source_link','types','user_pic']]

df

,scrape_date,date_update,provinsi,kabkot,kecamatan,kelurahan,total_odp,total_pdp,total_positif,positif_sembuh,positif_dirawat,positif_isolasi,positif_meninggal,source_link,types,user_pic
0,2021-12-02,2021-12-01,kepulauan bangka belitung,bangka,bakam,,,,161,0,0,0,8,http://covid19.bangka.go.id/#,kecamatan,Tubagus
1,2021-12-02,2021-12-01,kepulauan bangka belitung,bangka,belinyu,,,,1450,0,0,0,118,http://covid19.bangka.go.id/#,kecamatan,Tubagus
2,2021-12-02,2021-12-01,kepulauan bangka belitung,bangka,mendo barat,,,,729,0,0,0,24,http://covid19.bangka.go.id/#,kecamatan,Tubagus
3,2021-12-02,2021-12-01,kepulauan bangka belitung,bangka,merawang,,,,918,0,10,4,21,http://covid19.bangka.go.id/#,kecamatan,Tubagus
4,2021-12-02,2021-12-01,kepulauan bangka belitung,bangka,pemali,,,,1835,0,1,0,34,http://covid19.bangka.go.id/#,kecamatan,Tubagus
5,2021-12-02,2021-12-01,kepulauan bangka belitung,bangka,puding besar,,,,313,0,1,0,9,http://covid19.bangka.go.id/#,kecamatan,Tubagus
6,2021-12-02,2021-12-01,kepulauan bangka belitung,bangka,riau silip,,,,530,0,1,0,33,http://covid19.bangka.go.id/#,kecamatan,Tubagus
7,2021-12-02,2021-12-01,kepulauan bangka belitung,bangka,sungailiat,,,,3893,1,4,0,116,http://covid19.bangka.go.id/#,kecamatan,Tubagus


In [8]:
# Adding extra text for server
tempat = 'bangka'
tanggal = scrape_date

# Import into server
import mysql.connector
from mysql.connector import errorcode
from sqlalchemy import create_engine

try:
    mydb = mysql.connector.connect(host='db-blanja2',
                                  port=3306,
                                  user='covid_user',
                                  passwd='5bb6593aa078',
                                  database='covid')
    cursor = mydb.cursor()

    cols1 = "`,`".join([str(i) for i in df.columns.tolist()])
# Insert DataFrame records one by one. 
    for i,row in df.iterrows():
        sql = "INSERT INTO `covid19_data` (`" +cols1 + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)" 
        cursor.execute(sql, tuple(row)) 
# the connection is not autocommitted by default, so we must commit to save our # changes 
        mydb.commit()
#         print(now)

except mysql.connector.Error as error:
    print("Failed to insert into MySQL table {}".format(error))

finally:
    if (mydb.is_connected()):
        cursor.close()
        mydb.close()
#         print("MySQL connection is closed")
        print(tanggal + ' ' + tempat + ' Done')

import pymysql.cursors
import sqlalchemy

engine = sqlalchemy.create_engine('mysql+pymysql://covid_user5bb6593aa078@db-blanja2:3306/covid')

2021-09-21 bangka Done
